In [ ]:
import pandas as pd
import joblib
df = pd.read_csv("/Users/ryanghosh/Documents/Python <:>/phishing_email_detection_website/Phishing_Email.csv", dtype="str")
df2 = pd.read_csv("/Users/ryanghosh/Documents/Python <:>/phishing_email_detection_website/Enron.csv", dtype="str")

In [ ]:
x=df["Email Text"].fillna("")
y=df["Email Type"]
mask = (x.str.len() > 0) & (y.notna())
x = x[mask]
y = y[mask]
print(df.shape)

In [ ]:
df = pd.read_csv("/Users/ryanghosh/Documents/Python <:>/phishing_email_detection_website/Phishing_Email.csv", dtype="str")
df2 = pd.read_csv("/Users/ryanghosh/Documents/Python <:>/phishing_email_detection_website/Enron.csv", dtype="str")

print("Phishing_Email.csv shape:", df.shape)
print("Enron.csv shape:", df2.shape)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [ ]:
df2["subject"] = df2["subject"].fillna("")
df2["body"] = df2["body"].fillna("")

df2["text"] = df2["subject"] + " " + df2["body"]

x2 = df2["text"]
y2 = df2["label"]

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x2, y2, test_size=0.3, random_state=42
)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

svm_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english', max_features=5000)),
    ('classifier', LinearSVC())
])

svm_pipeline.fit(x_train, y_train)
preds = svm_pipeline.predict(x_test)
acc = accuracy_score(y_test, preds)

print("=== Results on first split (20% test) ===")
print(f"SVM Accuracy: {acc:.3f}")
print(classification_report(y_test, preds))

x2_train, x2_test, y2_train, y2_test = train_test_split(
    x2, y2, test_size=0.3, random_state=42
)

svm_pipeline.fit(x2_train, y2_train)
preds2 = svm_pipeline.predict(x2_test)
acc2 = accuracy_score(y2_test, preds2)

print("\n=== Results on second split (30% test) ===")
print(f"SVM Accuracy: {acc2:.3f}")
print(classification_report(y2_test, preds2))

joblib.dump(svm_pipeline, "svm_model.pkl")
print("saved as svm_model.pkl")


In [ ]:
import openai
openai.api_key = ""

def gpt_threat_analysis(email_text, prediction, confidence):
    prompt = f"""
    You are a cybersecurity assistant.

    An email has been classified by an ML phishing detector.

    Email content:
    {email_text}

    ML Prediction: {prediction} (confidence: {confidence:.2f})

    Provide:
    1. THREAT LEVEL: Low / Medium / High.
    2. Explanation of the potential RISKS if the user interacted with this email.
    3. Advice: Should the user BLOCK this sender or mark as safe?
    """

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": "You are a cybersecurity assistant."},
                  {"role": "user", "content": prompt}],
        max_tokens=300,
        temperature=0.3
    )

    return response["choices"][0]["message"]["content"]


In [ ]:
label_map = {
    '0': "safe",
    '1': "phishing"
}

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Predictions (you already have preds)
preds = svm_pipeline.predict(x_test)

# Create confusion matrix
cm = confusion_matrix(y_test, preds, labels=svm_pipeline.classes_)

# Display
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=svm_pipeline.classes_)
disp.plot(cmap="Blues", values_format='d')
plt.title("Confusion Matrix - SVM (20% Test Split)")
plt.show()


In [ ]:
for i, email in enumerate(x_test[:5]):  
    prediction = preds[i] # '0' or '1'
    human_label = label_map[prediction]

    print(f"\nEmail: {email[:200]}...")
    print(f"SVM Prediction: {human_label}")

    if human_label == "phishing":
        gpt_response = gpt_threat_analysis(email, human_label, acc)
        print("Analysis:\n", gpt_response)
    else:
        print("Safe email. No analysis needed.")
